# Pangeo Forge
## Transform large data archives into analysis-ready, cloud-optimized (ARCO) data stores

Presentation prepared by Charles Stern ([@cisaacstern](http://github.com/cisaacstern)), Data Infrastructure Engineer, Lamont-Doherty Earth Observatory (LDEO)

In [ ]:
import time
import pandas as pd
import matplotlib.pyplot as plt
from intake import open_catalog
from dask_gateway import Gateway
from dask.distributed import Client

plt.rcParams['figure.figsize'] = (15,10)
%matplotlib inline

In [ ]:
# I'll start and connect to this cluster ahead of time, but time it, so you'll know how long it took!
start = time.time()

gateway = Gateway()
cluster = gateway.new_cluster()
cluster.adapt(minimum=1, maximum=20)
client = Client(cluster)
print(f"Connected to Dask client in {round(time.time()-start, 2)} seconds")
client

## CMEMS sea surface altimetry data

For this example we will use [gridded sea-surface altimetry data](http://marine.copernicus.eu/services-portfolio/access-to-products/?option=com_csw&view=details&product_id=SEALEVEL_GLO_PHY_L4_REP_OBSERVATIONS_008_047) from The Copernicus Marine Environment. This is a widely used dataset in physical oceanography and climate. We will cover how to extracted the dataset from Copernicus store it in google cloud storage in [xarray-zarr](http://xarray.pydata.org/en/latest/io.html#zarr) format with the aid of [pangeo-forge-recipes](https://github.com/pangeo-forge/pangeo-forge-recipes).

## Familiarize ourselves with the `ftp` index

Opening `cmems_index/index.html` gives us a top level view of the `ftp` server directory structure, which looks something like this:

```
ftp://my.cmems-du.eu:21/Core/SEALEVEL_GLO_PHY_L4_REP_OBSERVATIONS_008_047/dataset-duacs-rep-global-merged-allsat-phy-l4/
 ├──/1993
 │   ├──/01
 │   │   ├── dt_global_allsat_phy_l4_19930101_20190101.nc  (7789577 bytes)
 │   │  ...
 │   │   └── dt_global_allsat_phy_l4_19930131_20190101.nc  (7853172 bytes)
 │  ...
 │   └──/12
...
 └──/2020
```

With a little help from `pandas`, we can see there's a total of 8901 files in our target range (January 1993 - mid March 2017):

In [ ]:
dates = pd.date_range(start="1993-01-01", end="2017-05-15")
len(dates)

### Start downloading (hint: this is going to take a while!)

The script `wget_cmems.py` will take care of this for us. Let's get it started.

# A better way: Pangeo Forge

`pangeo-forge-recipes` provides logic for transforming all of these

## What's a `recipe`?
A `recipe` is a Python file which can "see" all of the source files, and also knows how to logically arrange them into a cohesive dataset.

In [ ]:
from cmems_recipe import recipe

for time_index, url in recipe.file_pattern.items():
    if time_index[0] < 3 or time_index[0] > 8897:
        # note the '\033[...' are to color the output
        print(f"\033[1;32m{time_index}", f"\033[0m{url}")
    elif time_index[0] == 8897:
        print("...")
    else:
        pass

These are exactly the same files as we've begun downloading with `wget_cmems.py`, with the addition of these "alignment keys" (printed above in green) which allow `pangeo-forge-recipes` to not just *access* the source files, but also to **assemble** them into a cloud-optimized zarr store.

## Initialize Dataset

> In order to build a zarr store from the source files, the recipe must be "executed," which can happen in a number of ways. The most basic requirements for execution are a Python installation and an internet connection, but we're working on automating this process as we speak. 

I've already completed this build using `cmems_recipe.py`. From this point forward, getting up and running with the data is nearly instantaneous. There are a few options for loading the dataset from the zarr store, but using an Intake catalog is one of the easiest. Note that these very large datasets initialize nearly instantaneously, and we can see the full list of variables and coordinates, including metadata for each variable.

In [ ]:
cat = open_catalog("catalog.yaml")

for source in ["full_altimetry", "anomalies_only"]:
    ds = cat[source].to_dask()
    bold, reset = '\033[1m', '\033[0m'
    print(f"{bold}'{source}' is {round(ds.nbytes/1e9, 2)} GBs {reset}and contains {ds.data_vars} \n")

In [ ]:
ds

## Visually Examine Some of the Data

Let's do a sanity check that the data looks reasonable. Here we use the [hvplot](https://hvplot.holoviz.org/index.html) interactive plotting library.

In [ ]:
ds.sla.hvplot.image('longitude', 'latitude',
                    rasterize=True, dynamic=True, width=800, height=450,
                    widget_type='scrubber', widget_location='bottom', cmap='RdBu_r')

## Example calculation: timeseries of Global Mean Sea Level

Here we make a simple yet fundamental calculation: the rate of increase of global mean sea level over the observational period.

In [ ]:
# the number of GB involved in the reduction
ds.sla.nbytes/1e9

In [ ]:
# the computationally intensive step
sla_timeseries = ds.sla.mean(dim=('latitude', 'longitude')).load()

In [ ]:
sla_timeseries.plot(label='full data')
sla_timeseries.rolling(time=365, center=True).mean().plot(label='rolling annual mean')
plt.ylabel('Sea Level Anomaly [m]')
plt.title('Global Mean Sea Level')
plt.legend()
plt.grid()

Pause for discussion:

1. Wait a second, remember the download we started with `wget_cmems.py`?
   - We just did a lot of work analyzing a 73 GB subset of a 500+ GB dataset.
   - How much of that same dataset has been downloaded over these last few minutes?
   - Let's see ...
2. How do I make my own recipe?
   - Check out the docs: https://pangeo-forge.readthedocs.io/en/latest/
   - Reach out by opening an Issue here: https://github.com/pangeo-forge/staged-recipes/issues.
   - One prerequisite is that your dataset is in a format that can be opened with `xarray` (netCDF, etc.).
3. What's next for `pangeo-forge-recipes`?
   - Automated recipe execution in "Bakeries"
   - Cataloging with STAC
4. Your questions!
   - Reach out on GitHub if you think of them later :-)

Feel free to complete the additional computations and plots below on your own time. They're pretty interesting!

## Additional computations: Hovöller diagram & standard deviation

Here are some additional computations to run on the Dask cluster.

In order to understand how the sea level rise is distributed in latitude, we can make a sort of [Hovmöller diagram](https://en.wikipedia.org/wiki/Hovm%C3%B6ller_diagram).

In [ ]:
# another distributed computation
sla_hov = ds.sla.mean(dim='longitude').load()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))
sla_hov.name = 'Sea Level Anomaly [m]'
sla_hov.transpose().plot(vmax=0.2, ax=ax)

We can see that most sea level rise is actually in the Southern Hemisphere.

## Sea Level Variability

We can examine the natural variability in sea level by looking at its standard deviation in time.

In [ ]:
# one last computation
sla_std = ds.sla.std(dim='time').load()
sla_std.name = 'Sea Level Variability [m]'

In [ ]:
ax = sla_std.plot()
_ = plt.title('Sea Level Variability')